In [1]:
import numpy as np
import pandas as pd
import requests as rq
from io import BytesIO
from datetime import datetime, timedelta
import json
from tqdm import tqdm
import time
import pymysql
from sqlalchemy import create_engine
from dateutil.relativedelta import relativedelta
import re

C:\Users\YANG\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\YANG\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
biz_day = datetime.today() - timedelta(days=1)
biz_day = biz_day.strftime("%Y%m%d")

In [3]:
# url 입력
gen_otp_url = "http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd"
gen_otp_stk = {
    'mktId' : 'STK', # STK => 코스피, KSQ => 코스닥
    'trdDd' : biz_day, # 최근 영업일
    'money' : '1',
    'csvxls_isNo' : 'false',
    'name' : 'fileDown',
    'url' :'dbms/MDC/STAT/standard/MDCSTAT03901'
}

# 헤더 부분에 리퍼러를 추가한다. 리퍼러란 링크를 통해서 각각의 웹사잍로 방문할 때 남는 흔적이다.
# 헤더에 이러한 리퍼를 추가해야 로봇으로 인식하지 않는다.
headers = {'Referer' : 'http://dart.krx.co.kr/contents/MDC/MDI/mdiLoader'}
# post() 함수를 통해 해당 URL에 쿼리를 전송하면 이에 해당하는 데이터를 받으며, 이중 텍스트에 해당하는 내용만 불러온다.
opt_stk = rq.post(gen_otp_url, gen_otp_stk, headers=headers).text

# OTP를 제출할 URL을 down_url에 입력한다.
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'

# post() 함수를 통해 위에서 부여받은 OTP 코드를 해당 URL에 제출한다.
down_sector_stk = rq.post(down_url, {'code': opt_stk}, headers=headers)
# 받은 데이터의 content 부분을 BytesIO()를 이용해 바이너리 스트림 형태로 만든 후, csv 함수로 데이터를 읽어온다.
# 해당 데이터의 인코딩되어 있으므로 이를 선언한다.
sector_stk = pd.read_csv(BytesIO(down_sector_stk.content), encoding='EUC-KR')

In [4]:
gen_otp_ksq = {
    'mktId' : 'KSQ', # STK => 코스피, KSQ => 코스닥
    'trdDd' : biz_day, # 최근 영업일
    'money' : '1',
    'csvxls_isNo' : 'false',
    'name' : 'fileDown',
    'url' :'dbms/MDC/STAT/standard/MDCSTAT03901'
}

# 헤더 부분에 리퍼러를 추가한다. 리퍼러란 링크를 통해서 각각의 웹사잍로 방문할 때 남는 흔적이다.
# 헤더에 이러한 리퍼를 추가해야 로봇으로 인식하지 않는다.
headers = {'Referer' : 'http://dart.krx.co.kr/contents/MDC/MDI/mdiLoader'}
# post() 함수를 통해 해당 URL에 쿼리를 전송하면 이에 해당하는 데이터를 받으며, 이중 텍스트에 해당하는 내용만 불러온다.
opt_ksq = rq.post(gen_otp_url, gen_otp_ksq, headers=headers).text

# OTP를 제출할 URL을 down_url에 입력한다.
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'

# post() 함수를 통해 위에서 부여받은 OTP 코드를 해당 URL에 제출한다.
down_sector_ksq = rq.post(down_url, {'code': opt_ksq}, headers=headers)
# 받은 데이터의 content 부분을 BytesIO()를 이용해 바이너리 스트림 형태로 만든 후, csv 함수로 데이터를 읽어온다.
# 해당 데이터의 인코딩되어 있으므로 이를 선언한다.
sector_ksq = pd.read_csv(BytesIO(down_sector_ksq.content), encoding='EUC-KR')

In [5]:
# 두개의 데이터를 합침
krx_sector = pd.concat([sector_stk, sector_ksq]).reset_index(drop=True)
krx_sector['종목명'] = krx_sector['종목명'].str.strip()
krx_sector['기준일'] = biz_day

In [6]:
gen_otp_data = {
    'searchType' : '1',
    'mktId' : 'ALL', # STK => 코스피, KSQ => 코스닥
    'trdDd' : biz_day, # 최근 영업일
    'csvxls_isNo' : 'false',
    'name' : 'fileDown',
    'url' :'dbms/MDC/STAT/standard/MDCSTAT03501'
}

headers = {'Referer' : 'http://dart.krx.co.kr/contents/MDC/MDI/mdiLoader'}
otp = rq.post(gen_otp_url, gen_otp_data, headers=headers).text
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
krx_ind = rq.post(down_url, {'code' : otp}, headers=headers)

krx_ind = pd.read_csv(BytesIO(krx_ind.content), encoding = 'EUC-KR')
krx_ind['종목명'] = krx_ind['종목명'].str.strip()
krx_ind['기준일'] = biz_day

In [7]:
# symmetric_difference 대칭차 구하는 함수
diff = list(set(krx_sector['종목명']).symmetric_difference(set(krx_ind['종목명'])))

In [8]:
# on 조건을 기준으로 두 데이터를 하나로 합치며, intersection() 메서드를 이용해 공통으로 존재하는 열을 기준으로 입력한다
kor_ticker = pd.merge(krx_sector, krx_ind, on=krx_sector.columns.intersection(krx_ind.columns).tolist(), how='outer')

In [9]:
# 스팩 종목은 종목명에 스팩 또는 제n호 단어가 들어간다.
# print(kor_ticker[kor_ticker['종목명'].str.contains('스팩|제[0-9]+호')]['종목명'].values)

In [10]:
# 국내 종목 중 종목코드 끝이 0이 아닌 종목은 우선주에 해당한다.
# print(kor_ticker[kor_ticker['종목코드'].str[-1:] != '0']['종목명'].values)

In [11]:
# 리츠 종목은 종목명이 리츠로 끝난다
# print(kor_ticker[kor_ticker['종목명'].str.endswith('리츠')]['종목명'].values)

In [12]:
kor_ticker['종목구분'] = np.where(kor_ticker['종목명'].str.contains('스팩|제[0-9]+호'), '스팩', np.where(kor_ticker['종목코드'].str[-1:] != '0', '우선주',
                        np.where(kor_ticker['종목명'].str.endswith('리츠'), '리츠', np.where(kor_ticker['종목명'].isin(diff), '기타', '보통주'))))
kor_ticker = kor_ticker.reset_index(drop=True)
kor_ticker.columns = kor_ticker.columns.str.replace(' ', '')
kor_ticker = kor_ticker[['종목코드', '종목명', '시장구분', '종가', '시가총액', '기준일', 'EPS', '선행EPS', 'BPS', '주당배당금', '종목구분']]
kor_ticker = kor_ticker.replace({np.nan: None})
kor_ticker['기준일'] = pd.to_datetime(kor_ticker['기준일'])

In [91]:
# # initial database
# con = pymysql.connect(user='root',
#                       passwd='Tjdgus12!',
#                       host='127.0.0.1',
#                       db='kr_stock_db',
#                       charset = 'utf8')
# mycursor = con.cursor()
# query = f"""
# insert into kor_ticker (종목코드,종목명,시장구분,종가,시가총액,기준일,EPS,선행EPS,BPS,주당배당금,종목구분)
# values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) as new
# on duplicate key update
# 종목명 = new.종목명, 시장구분 = new.시장구분, 종가 = new.종가, 시가총액 = new.시가총액,
#  EPS = new.EPS, 선행EPS = new.선행EPS, BPS = new.BPS, 주당배당금 = new.주당배당금, 종목구분 = new.종목구분;
# """

# args = kor_ticker.values.tolist()
# mycursor.executemany(query, args)
# con.commit()
# con.close()

In [20]:
# update database
con = pymysql.connect(user='root',
                      passwd='Tjdgus12!',
                      host='127.0.0.1',
                      db='kr_stock_db',
                      charset='utf8')
mycursor = con.cursor()

query = """
UPDATE kor_ticker
SET 종목명 = %s, 시장구분 = %s, 종가 = %s,
    시가총액 = %s, 기준일 = %s, EPS = %s, 선행EPS = %s,
    BPS = %s, 주당배당금 = %s, 종목구분 = %s
WHERE 종목코드 = %s;
"""

args = kor_ticker[['종목명', '시장구분', '종가', '시가총액', '기준일', 'EPS', '선행EPS', 'BPS', '주당배당금', '종목구분', '종목코드']].values.tolist()
mycursor.executemany(query, args)
con.commit()
con.close()


WICS 기준 섹터 정보 크롤링  
WICS(Wise Industry Classification Standard)는 당사에서 사용하는 대표적인 Sector 분류기준으로, 국제적으로 통용되는 분류 기준을 국내 실정에 맞게 재구성하여 확립하였습니다.  
https://www.wiseindex.com/About/WICS



In [55]:
url = f'''https://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt=20240319&sec_cd=G10'''
data = rq.get(url).json()
type(data)

dict

In [43]:
data['sector']

[{'SEC_CD': 'G25', 'SEC_NM_KOR': '경기관련소비재', 'SEC_RATE': 10.07, 'IDX_RATE': 0},
 {'SEC_CD': 'G35', 'SEC_NM_KOR': '건강관리', 'SEC_RATE': 9.87, 'IDX_RATE': 0},
 {'SEC_CD': 'G50', 'SEC_NM_KOR': '커뮤니케이션서비스', 'SEC_RATE': 5.77, 'IDX_RATE': 0},
 {'SEC_CD': 'G40', 'SEC_NM_KOR': '금융', 'SEC_RATE': 9.13, 'IDX_RATE': 0},
 {'SEC_CD': 'G10', 'SEC_NM_KOR': '에너지', 'SEC_RATE': 1.7, 'IDX_RATE': 100.0},
 {'SEC_CD': 'G20', 'SEC_NM_KOR': '산업재', 'SEC_RATE': 12.61, 'IDX_RATE': 0},
 {'SEC_CD': 'G55', 'SEC_NM_KOR': '유틸리티', 'SEC_RATE': 1.01, 'IDX_RATE': 0},
 {'SEC_CD': 'G30', 'SEC_NM_KOR': '필수소비재', 'SEC_RATE': 1.96, 'IDX_RATE': 0},
 {'SEC_CD': 'G15', 'SEC_NM_KOR': '소재', 'SEC_RATE': 8.62, 'IDX_RATE': 0},
 {'SEC_CD': 'G45', 'SEC_NM_KOR': 'IT', 'SEC_RATE': 39.26, 'IDX_RATE': 0}]

In [44]:
biz_day

'20240320'

In [67]:
data_pd = pd.json_normalize(data['list'])
sector_code = ['G25', 'G35','G50', 'G40', 'G10', 'G20', 'G55', 'G30', 'G15', 'G45']
data_sector = []

for i in tqdm(sector_code):
    url = f'''https://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt={biz_day}&sec_cd={i}'''
    data = rq.get(url).json()
    data_pd = pd.json_normalize(data['list'])
    data_sector.append(data_pd)
    time.sleep(2)

kor_sector = pd.concat(data_sector, axis = 0)
kor_sector = kor_sector[['IDX_CD', 'CMP_CD', 'CMP_KOR', 'SEC_NM_KOR']]
kor_sector['기준일'] = biz_day
kor_sector['기준일'] = pd.to_datetime(kor_sector['기준일'])

  0%|          | 0/10 [00:00<?, ?it/s]

https://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt=20240319&sec_cd=G25


 10%|█         | 1/10 [00:02<00:20,  2.28s/it]

https://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt=20240319&sec_cd=G35


 20%|██        | 2/10 [00:04<00:17,  2.22s/it]

...

 80%|████████  | 8/10 [00:17<00:04,  2.14s/it]

https://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt=20240319&sec_cd=G15


 90%|█████████ | 9/10 [00:19<00:02,  2.15s/it]

https://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt=20240319&sec_cd=G45


100%|██████████| 10/10 [00:21<00:00,  2.18s/it]


In [69]:
con = pymysql.connect(user='root',
                      passwd='Tjdgus12!',
                      host='127.0.0.1',
                      db='kr_stock_db',
                      charset = 'utf8')
mycursor = con.cursor()
query = f"""
insert into kor_sector (IDX_CD, CMP_CD, CMP_KOR, SEC_NM_KOR, 기준일)
values(%s,%s,%s,%s,%s) as new
on duplicate key update
IDX_CD = new.IDX_CD, CMP_CD = new.CMP_CD, CMP_KOR = new.CMP_KOR, SEC_NM_KOR = new.SEC_NM_KOR
"""

args = kor_sector.values.tolist()
mycursor.executemany(query, args)
con.commit()
con.close()